<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/OpenAir1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###############################################################
# ANÀLISI DE LA CONTAMINACIÓ DE L'AIRE A MARTORELL (1991-2025)
# NIVELL: 4t ESO -DIGITALITZACiÓ- BATXILLERAT – PROGRAMACIÓ
#
# OBJECTIUS DEL CODI:
# - Importar dades de contaminació atmosfèrica
# - Preparar les dates correctament
# - Analitzar variacions temporals dels contaminants
# - Visualitzar episodis de contaminació
# - Detectar superacions de la normativa europea
#
# NORMATIVA AMBIENTAL UTILITZADA:
# Directiva 2008/50/CE de qualitat de l'aire de la Unió Europea
#
# LÍMITS PRINCIPALS:
# NO2:
#   - 200 µg/m³ límit horari
#   - 40 µg/m³ límit anual
#
# PM10:
#   - 50 µg/m³ límit diari (màxim 35 dies/any)
#   - 40 µg/m³ límit anual
###############################################################

##############################
# 1. CARREGAR LLIBRERIES
##############################

# openair és una llibreria especialitzada en anàlisi de qualitat de l'aire
library(openair)

##############################
# 2. IMPORTAR DADES
##############################

# Llegim el fitxer de dades ambientals
# Cada columna representa un contaminant atmosfèric
ciutatwide <- read.csv("processed_data_wide.csv")

##############################
# 3. PREPARAR LA COLUMNA DE DATA
##############################

# Convertim la columna "date" de text a format temporal real
# Això permet fer anàlisis per hores, dies, mesos i anys
ciutatwide$date <- as.POSIXct(
  ciutatwide$date,
  "%Y-%m-%d %H:%M:%S",
  tz = "Europe/Madrid"
)

##############################
# 4. VISUALITZACIÓ GENERAL DE DADES
##############################

# Mostra la taula de dades (opcional en entorn gràfic)
View(ciutatwide)

##############################
# 5. VARIACIÓ TEMPORAL GLOBAL
##############################

# Analitza com varien els contaminants:
# - al llarg del dia
# - durant la setmana
# - durant l'any
# - al llarg de dècades

timeVariation(
  ciutatwide,
  pollutant = c(
    "co","h2s","hcnm","hct",
    "no","no2","nox","o3",
    "pm10","pm2.5","so2"
  ),
  main = "Contaminació de l'aire a Martorell (1991-2025)"
)

##############################
# 6. FILTRAR DADES DE L'ANY 2025
##############################

# Seleccionem només registres de l'any 2025
dades_2025 <- subset(
  ciutatwide,
  format(date, "%Y") == "2025"
)

##############################
# 7. VARIACIÓ TEMPORAL 2025
##############################

# Analitza els principals contaminants relacionats amb el trànsit
timeVariation(
  dades_2025,
  pollutant = c("no","no2","nox","o3","pm10","pm2.5"),
  main = "Contaminació de l'aire a Martorell (2025)"
)

##############################
# 8. CALENDAR PLOT – PM10
##############################

# Representa cada dia com una casella de calendari
# Permet detectar episodis puntuals de contaminació
calendarPlot(
  dades_2025,
  pollutant = "pm10",
  year = 2025,
  limits = c(0,80),
  main = "PM10 – Martorell (2025)"
)

##############################
# 9. SUPERACIONS DIÀRIES PM10
##############################

# Límit legal europeu PM10 diari:
# 50 µg/m³
calendarPlot(
  dades_2025,
  pollutant = "pm10",
  year = 2025,
  exceedance = 50,
  main = "PM10 – Superacions del límit diari (50 µg/m³)"
)

##############################
# 10. CALENDAR AMB RANGS DE QUALITAT
##############################

calendarPlot(
  dades_2025,
  pollutant = "pm10",
  year = 2025,
  limits = c(0,25,50,150),
  cols = c("green","orange","red"),
  main = "PM10 – Verd <25 | Taronja 25-50 | Vermell >50"
)

##############################
# 11. TENDÈNCIA HISTÒRICA NO2
##############################

# Permet veure si la contaminació augmenta o disminueix
trendLevel(
  ciutatwide,
  pollutant = "no2",
  main = "Tendència anual de NO2 (1991-2025)"
)

##############################
# 12. CÀLCUL DE SUPERACIONS LEGALS
##############################

### NO2 – LÍMIT HORARI
# 200 µg/m³
no2_hourly_exceed <- sum(
  ciutatwide$no2 > 200,
  na.rm = TRUE
)

cat("Hores amb NO2 > 200 µg/m³:", no2_hourly_exceed, "\n")

### NO2 – LÍMIT ANUAL
# 40 µg/m³
no2_annual <- timeAverage(
  ciutatwide,
  avg.time = "year",
  pollutant = "no2"
)

no2_annual_exceed <- sum(
  no2_annual$no2 > 40,
  na.rm = TRUE
)

cat("Anys amb NO2 > 40 µg/m³:", no2_annual_exceed, "\n")

##############################
# 13. PM10 – MITJANES DIÀRIES
##############################

pm10_daily <- timeAverage(
  ciutatwide,
  avg.time = "day",
  pollutant = "pm10"
)

pm10_daily_exceed <- sum(
  pm10_daily$pm10 > 50,
  na.rm = TRUE
)

cat("Dies amb PM10 > 50 µg/m³:", pm10_daily_exceed, "\n")

##############################
# 14. PM10 – MITJANES ANUALS
##############################

pm10_annual <- timeAverage(
  ciutatwide,
  avg.time = "year",
  pollutant = "pm10"
)

pm10_annual_exceed <- sum(
  pm10_annual$pm10 > 40,
  na.rm = TRUE
)

cat("Anys amb PM10 > 40 µg/m³:", pm10_annual_exceed, "\n")

##############################
# 15. IDENTIFICAR EPISODIS CONCRETS
##############################

# Dies concrets amb contaminació elevada de PM10
pm10_exceed_days <- subset(
  pm10_daily,
  pm10 > 50
)

pm10_exceed_days[,c("date","pm10")]

# Hores concretes amb NO2 elevat
no2_exceed_hours <- subset(
  ciutatwide,
  no2 > 200
)

no2_exceed_hours[,c("date","no2")]

# Anys amb excés anual de NO2
no2_annual_exceed <- subset(
  no2_annual,
  no2 > 40
)

no2_annual_exceed[,c("date","no2")]


##############################
# 15b. TAULA COMPLETA DE SUPERACIONS (1991-2021)
##############################

# Límits legals segons Directiva UE 2008/50/CE
limits <- list(
  no2_hour = 200,    # µg/m³
  no2_year = 40,     # µg/m³
  pm10_day = 50,     # µg/m³
  pm10_year = 40,    # µg/m³
  pm25_day = 25,     # µg/m³ nou límit UE 2024, s'integra en anual
  pm25_year = 10,    # µg/m³
  o3_hour8 = 120,    # µg/m³ valor objectiu UE 2024
  o3_warm = 60,      # µg/m³ pic 6 mesos OMS 2021
  so2_hour = 350,    # µg/m³
  so2_day = 125,     # µg/m³
  so2_year = 20,     # µg/m³
  co_hour8 = 10,     # mg/m³
  co_day = 10,       # mg/m³, 24h OMS
  benzene_year = 5,  # µg/m³ històric UE
  pb_year = 0.5      # µg/m³
)

# Filtrar dades 1991-2021
dades_period <- subset(
  ciutatwide,
  format(date, "%Y") >= "1991" & format(date, "%Y") <= "2021"
)

# Inicialitzem taula buida
superacions <- data.frame(
  contaminant = character(),
  tipus_limit = character(),
  data = as.POSIXct(character()),
  valor_mesurat = numeric(),
  stringsAsFactors = FALSE
)

##############################
# FUNCIONS AUXILIARS
##############################

# Afegir superacions horàries
add_hourly_exceed <- function(contaminant, limit){
  subset_data <- subset(dades_period, dades_period[[contaminant]] > limit)
  if(nrow(subset_data) > 0){
    tmp <- data.frame(
      contaminant = contaminant,
      tipus_limit = "hora",
      data = subset_data$date,
      valor_mesurat = subset_data[[contaminant]]
    )
    return(tmp)
  } else {
    return(NULL)
  }
}

# Afegir superacions diàries
add_daily_exceed <- function(contaminant, limit){
  daily_avg <- timeAverage(dades_period, avg.time = "day", pollutant = contaminant)
  subset_data <- subset(daily_avg, daily_avg[[contaminant]] > limit)
  if(nrow(subset_data) > 0){
    tmp <- data.frame(
      contaminant = contaminant,
      tipus_limit = "dia",
      data = subset_data$date,
      valor_mesurat = subset_data[[contaminant]]
    )
    return(tmp)
  } else {
    return(NULL)
  }
}

# Afegir superacions anuals
add_yearly_exceed <- function(contaminant, limit){
  yearly_avg <- timeAverage(dades_period, avg.time = "year", pollutant = contaminant)
  subset_data <- subset(yearly_avg, yearly_avg[[contaminant]] > limit)
  if(nrow(subset_data) > 0){
    tmp <- data.frame(
      contaminant = contaminant,
      tipus_limit = "any",
      data = subset_data$date,
      valor_mesurat = subset_data[[contaminant]]
    )
    return(tmp)
  } else {
    return(NULL)
  }
}

##############################
# 15b. SUPERACIONS PER CADA CONTAMINANT
##############################

# NO2
superacions <- rbind(superacions, add_hourly_exceed("no2", limits$no2_hour))
superacions <- rbind(superacions, add_yearly_exceed("no2", limits$no2_year))

# PM10
superacions <- rbind(superacions, add_daily_exceed("pm10", limits$pm10_day))
superacions <- rbind(superacions, add_yearly_exceed("pm10", limits$pm10_year))

# PM2.5
superacions <- rbind(superacions, add_daily_exceed("pm2.5", limits$pm25_day))
superacions <- rbind(superacions, add_yearly_exceed("pm2.5", limits$pm25_year))

# SO2
superacions <- rbind(superacions, add_hourly_exceed("so2", limits$so2_hour))
superacions <- rbind(superacions, add_daily_exceed("so2", limits$so2_day))
superacions <- rbind(superacions, add_yearly_exceed("so2", limits$so2_year))

# CO
superacions <- rbind(superacions, add_hourly_exceed("co", limits$co_hour8))
superacions <- rbind(superacions, add_daily_exceed("co", limits$co_day))

# O3 (8h)
superacions <- rbind(superacions, add_hourly_exceed("o3", limits$o3_hour8))

# O3 temporada càlida (OMS 2021)
# Assumim columna "o3_warm" amb mitjana de 6 mesos calents
if("o3_warm" %in% names(dades_period)){
  superacions <- rbind(superacions, add_yearly_exceed("o3_warm", limits$o3_warm))
}

# Benzè anual
superacions <- rbind(superacions, add_yearly_exceed("benzene", limits$benzene_year))

# Plom anual
superacions <- rbind(superacions, add_yearly_exceed("pb", limits$pb_year))

##############################
# ORDENAR TAULA
##############################
superacions <- superacions[order(superacions$contaminant,
                                 superacions$tipus_limit,
                                 superacions$data),]

# Mostrar taula completa
View(superacions)

# Exportar a CSV
write.csv(superacions, "superacions_completes_1991_2021.csv", row.names = FALSE)

cat("Taula completa de superacions generada amb", nrow(superacions), "registre(s)\n")


#####################################################
# FINAL DEL SCRIPT
###############################################################